<h1>ELECTRE Tri-B analysis with default parameters: example of a building energy retrofit</h1>

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/cghiaus/ELECTRE_Tri/HEAD?labpath=docs%2Ftutorials%2Fbldg_retrofit_defaults.ipynb)

In the case of energy retrofit of a building, let's suppose that there are three alternatives:
- basic renovation,
- moderate renovation,
- extensive renovation,

each one yielding energy savings and requiring costs. 

We want to sort these three alternatives into three categories (e.g. bad, acceptable, good) based on their performance on criteria of energy savings and costs. The categories are defined by equidistant base profiles between  two extreme levels.

See [/explanation/electre_tri-b_explained.ipynb](../explanation/electre_tri-b_explained.ipynb#Definitions) for definitions of concepts.

**Given:**

- A set of criteria $\{c_{k}\}$:
    - __c1: Saving (in kWh/m²/year)__ with weight $w = 0.7$
    - __c2: Cost (in €/m²)__ with weight $w = 0.3$
___
- A set of alternatives $\{a_{i}\}$:
   - __a1: Basic renovation__
   - __a2: Moderate renovation__
   - __a3: Extensive renovation__
___
- A matrix $A = \{a_{i,k}\}$ of performances of alternatives $a_i$ for each criterion $c_k$.

| Alternative profile | c1: Saving/(kWh/m²/year)| c2: Cost/(€/m²)|
|---------------------------|------------------:|------------:|
| a1: Basic renovation      | 50.0              | 100.0       |
| a2: Moderate renovation   | 80.0              | 200.0       |
| a3: Extensive renovation  | 120.0             | 350.0       |
___

- A matrix $L = \{b_{j,k}\}$ of extreme levels of the base profiles for each criterion $c_k$.

| Level profile | c1: Saving/(kWh/m²/year) | c2: Cost/(€/m²) |
|---------------|---------------------:|-------------:|
| worst         | 0.0                  |   300.0      |
| best          | 150.0                |   100.0      |

These base profiles are the lowest and higest limit of five equidistant categories:

- __b1 >__ worse than the worst profile __b1__;
- __(b1, b2)__ between base profiles __b1__ and __b2__;
- __(b2, b3)__ between base profiles __b2__ and __b3__;
- __(b3, b4)__ between base profiles __b3__ and __b4__;  
- __b4 <__ better than the best profile __b4__.
____

- The thresholds are:
  - indifference $q = \{q_k\}$, the largest difference in performance on a criterion that a decision-maker considers insignificant;
  - preference $p = \{p_k\}$,  the smallest difference in performance on a criterion that the decision-maker considers significant enough to strongly prefer one alternative over another;
  - veto $v = \{v_k\}$, the maximum allowable difference in performance on a criterion beyond which an alternative cannot outrank another, regardless of its performance on other criteria.

The default values of the thresholds are:

$q = 10 \,\%$, $p = 25 \,\%$, $v = 50 \,\%$
of the range of the equidistant categories.
___

- A credibility threshold $\lambda$ for outranking, i.e. minimum degree of credibility index that is considered necessary to validate the statement "alternative a outranks base profile b" (value within the range [0.5, 1], typically 0.75).
____

**Do:**

Assign each alternative $a_i ∈ A$ to one of the predefined categories $C_j ∈ C$ based on its performance across all criteria $c_k$ by using optimistic and pessimistic classification:

| Categories   | Alternatives                             |
|--------------|------------------------------------------|
| __b1 ≻__     |                                          |
| __(b1, b2)__ | [list of alternatives between b1 and b2] |
| __(b2, b3)__ | [list of alternatives between b2 and b3] |
| __(b3, b4)__ | [list of alternatives between b3 and b4] |
| __b4 ≺__     |                                          |

____

__Procedure:__
1. Upload your data file containing the performance of alternatives, base profiles, thresholds, and weights.
2. Read data file and solve ELECTRE Tri-B sorting problem.
3. Interpret the results.

In [1]:
"""
Append `src/` directory to `path`
"""
import sys
import os

notebook_dir = os.path.dirname(os.path.abspath('__file__'))
project_root = os.path.dirname(os.path.dirname(notebook_dir))

src_dir = os.path.join(project_root, 'src')
sys.path.append(src_dir)

In [2]:
import pandas as pd

import electre_tri as et

# Data file

ELECTRE Tri-B data is a [.csv](https://en.m.wikipedia.org/wiki/Comma-separated_values) text file that contains, for each criterion:
- performance of alternatives **A**;
- extreme level profiles **L**;
- weights **w**. 

__Note__: For criteria to be minimized (i.e. cost), the data for performance of alternatives __A__ and the base profiles __L__ are negative numbers.

Data files can be written by using a [spreadsheet](https://en.m.wikipedia.org/wiki/Spreadsheet).

In [3]:
data_file = "../../data/bldg_retrofit_levels.csv"
print("Example of data file")
pd.read_csv(data_file)

Example of data file


,type,profile,Saving/(kWh/m²/year),Cost/(€/m²)
0,A,a1: Basic renovation,50.0,-100.0
1,A,a2: Moderate renovation,80.0,-200.0
2,A,a3: Extensive renovation,120.0,-350.0
3,L,worst,0.0,-400.0
4,L,best,150.0,-50.0
5,w,NaN,0.7,0.3


# Problem solving

In [5]:
# Problem statement
data_file = "../../data/bldg_retrofit_levels.csv"
credibility_threshold = 0.7

# Problem solving
A, L, w = et.read_electre_tri_extreme_base_profile(data_file)
B = et.base_profile(L)
T = et.threshold(B)

optimistic, pessimistic = et.electre_tri_b(A, B, T, w,
                                           credibility_threshold)

# Results

In [6]:
# Optimistic sorting
opti_sort = et.sort(optimistic)
print('Optimistic sorting')
opti_sort.to_frame(name="alternatives").rename_axis("categories")

Optimistic sorting


,alternatives
categories,
b1 ≻,[]
"(b1, b2)",[]
"(b2, b3)",[a2: Moderate renovation]
"(b3, b4)","[a1: Basic renovation, a3: Extensive renovation]"
b4 ≺,[]


In [7]:
# Pessimistic sorting
pessi_sort = et.sort(pessimistic)
print('Pessimistic sorting')
pessi_sort.to_frame(name="alternatives").rename_axis("categories")

Pessimistic sorting


,alternatives
categories,
b1 ≻,[]
"(b1, b2)","[a1: Basic renovation, a3: Extensive renovation]"
"(b2, b3)",[a2: Moderate renovation]
"(b3, b4)",[]
b4 ≺,[]
